### Analisis exploratorio

Instalar y cargar librarias

In [1]:
import numpy as np
import pandas as pd 
import sqlite3 as sql

In [2]:
#Realizar conexión de base de datos db_movies
conn=sql.connect('BD_insurance2019dataset.db')
cur=conn.cursor()

In [3]:
#Verificar las tablas existentes en db_movies
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('reclamaciones',),
 ('diagnosticos',),
 ('regional',),
 ('genero',),
 ('sociodemograficas',),
 ('utilizaciones',)]

In [5]:
pd.read_sql_query("SELECT name FROM sqlite_master;",conn)

,name
0,reclamaciones
1,diagnosticos
2,regional
3,genero
4,sociodemograficas
5,utilizaciones
